# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df = pd.DataFrame(cities)
cities_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Temperature (F)
0,Norman Wells,65.2820,-126.8329,286.17,62,1,5.14,CA,1623854263,55.71
1,Victoria Point,-27.5833,153.3000,292.14,95,100,0.45,AU,1623854264,66.45
2,Bluff,-46.6000,168.3333,280.26,80,99,5.48,NZ,1623854264,45.07
3,Tretten,61.3142,10.3007,291.70,48,100,4.12,NO,1623854264,65.66
4,Grand-Santi,4.2500,-54.3833,301.20,79,69,1.73,GF,1623854265,82.76
...,...,...,...,...,...,...,...,...,...,...
551,Rjukan,59.8789,8.5941,294.10,47,100,6.47,NO,1623854469,69.98
552,Zhigansk,66.7697,123.3711,287.29,64,0,3.09,RU,1623854470,57.72
553,Iquitos,-3.7481,-73.2472,298.17,88,75,1.54,PE,1623854470,77.31
554,Ranfurly,-45.1333,170.1000,278.51,98,97,2.16,NZ,1623854471,41.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#storing locaiton and humidity values for a heat map
location_df = cities_df[["Lat", "Lng"]]
humidity_df = cities_df["Humidity"]

In [9]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [162]:
figure_layout = {
    'width': '9000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,center=(46.0, -5.0), zoom_level=2)
heatmap = gmaps.heatmap_layer(location_df, weights=humidity_df, point_radius = 15)
fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [40]:
max_temp = 80
min_temp = 70
wind_speed = 10
cloud = 0

cities_filtered_df = cities_df.loc[(cities_df["Temperature (F)"] > min_temp) & (cities_df["Temperature (F)"] < max_temp) & (cities_df["Wind Speed"] < wind_speed) & (cities_df["Cloudiness"] == cloud),: ]
cities_filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Temperature (F)
127,Visby,57.6409,18.2960,294.85,52,0,5.14,SE,1623854309,71.33
251,Sirte,31.2089,16.5887,298.12,67,0,6.86,LY,1623854354,77.22
365,Yabrūd,33.9674,36.6574,297.96,24,0,1.50,SY,1623854396,76.93
375,Kimberley,-28.7323,24.7623,295.98,20,0,6.69,ZA,1623854400,73.36
410,Pimentel,-6.8367,-79.9342,294.31,78,0,4.63,PE,1623854415,70.36
440,Oryol,52.9658,36.0803,294.64,77,0,5.08,RU,1623854161,70.95
467,Upington,-28.4478,21.2561,295.36,30,0,3.60,ZA,1623854439,72.25


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [132]:
hotel_df = cities_filtered_df[['Lat','Lng',"City","Country"]]
hotel_df['Hotel Name'] = ""
hotel_df

/opt/anaconda3/envs/AKpython/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Lat,Lng,City,Country,Hotel Name
127,57.6409,18.2960,Visby,SE,
251,31.2089,16.5887,Sirte,LY,
365,33.9674,36.6574,Yabrūd,SY,
375,-28.7323,24.7623,Kimberley,ZA,
410,-6.8367,-79.9342,Pimentel,PE,
440,52.9658,36.0803,Oryol,RU,
467,-28.4478,21.2561,Upington,ZA,


In [133]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {    
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

for index,row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    #get locations from hotel_df
    params['location'] = f"{lat},{lng}"
    
    #add location to params
    #params['location'] = location

    response = requests.get(base_url, params=params).json()
    
    # extract results
    
    #print(response['results'][0]['name'])
    
    hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    
hotel_df


/opt/anaconda3/envs/AKpython/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Lat,Lng,City,Country,Hotel Name
127,57.6409,18.2960,Visby,SE,Best Western Solhem Hotel Visby
251,31.2089,16.5887,Sirte,LY,الهادي ابوشعاله سرت
365,33.9674,36.6574,Yabrūd,SY,صالون مها حيدر - Salon Maha
375,-28.7323,24.7623,Kimberley,ZA,Welgewandel Guesthouse
410,-6.8367,-79.9342,Pimentel,PE,Casona Del Muelle
440,52.9658,36.0803,Oryol,RU,Атлантида
467,-28.4478,21.2561,Upington,ZA,River Place Manor


In [134]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [156]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}



fig = gmaps.figure(layout=figure_layout,center=(46.0, -5.0), zoom_level=2)


#print(fig)
#heatmap = gmaps.heatmap_layer(location_df, weights=humidity_df, point_radius = 15)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(heatmap)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…